<a href="https://colab.research.google.com/github/borlaugg/Improving-Bilateral-Filters/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
from numpy.linalg import matrix_power
import matplotlib.pyplot as plt

In [67]:
import PIL
from PIL import Image
from PIL import ImageOps
from numpy import asarray
Z = asarray(ImageOps.grayscale(Image.open('img.jpg')))
Z = Z[380:680,400:700]
print(Z.shape)
Z = Z.astype('float64')
Y = Z + np.random.normal(size=Z.shape,scale  = 50, loc = 0)
img = Image.fromarray(Y)
img.show()

(300, 300)


In [68]:
# @title
## Constants
sigma_s = 2.5
sigma_r = 0.5
N = Y.shape[0]
N0 = 10
iter = 100
I = np.eye(N)
D_h = np.roll(I,1,axis = 0)
D_v = np.transpose(D_h)
E_h = np.linalg.pinv(D_h)
E_v = np.linalg.pinv(D_v)
W = I
# Y = np.random.normal(size=(N,N))
lamda = 15
zeta = 0.0001
alpha = 0.2
e = 0.5

In [69]:
def W(Y,n,D):
  A = np.eye(N)*alpha**n
  #A = (I - matrix_power(D,n))@Y*alpha**n
  #A = np.divide(np.ones(I.shape), np.sqrt( np.square(Y) + e**2))*alpha**n
  return A

In [70]:
def SD_h(Y, N, zeta,E_h,D_h):
  prev_X = Y
  sum = np.zeros((N,N))
  for n in range(1,3):
      sum += (I - matrix_power(E_h, n))@W(Y,n,D_h)@(I - matrix_power(D_h,n))
  H = I + lamda*sum
  M = np.linalg.pinv(zeta*I + np.diag(np.diag(H)))
  for i in range(1,iter):
    slope = (I + lamda*sum)@prev_X - Y
    X = prev_X - M@slope
    prev_X = X
  return prev_X

def SD_v(Y, N, zeta,E_v,D_v):
  prev_X = Y
  sum = np.zeros((N,N))
  for n in range(1,N0):
      sum += (I - matrix_power(E_v, n))@W(Y,n,D_v)@(I - matrix_power(D_v,n))
  H = I + lamda*sum
  M = np.linalg.pinv(zeta*I + np.diag(np.diag(H)))
  for i in range(1,iter):
    slope = (I + lamda*sum)@prev_X - Y
    X = prev_X - M@slope
    prev_X = X
  return prev_X


In [71]:
X_bi = SD_h(Y,N,zeta,E_h,D_h) ##Regular bilateral Filter
img = Image.fromarray((X))
img.show()

In [72]:
X = SD_h(Y,N,zeta,E_h,D_h)
#X = SD_v(np.transpose(X),N,zeta,E_v,D_v)
X = SD_h(np.transpose(X),N,zeta,E_h,D_h) #this is what lead to the idea
img = Image.fromarray(np.transpose(X))
img.show()

In [73]:
i = Image.fromarray(Z)
i.show()

In [74]:
alpha = 0.83
N1 = 3
def SD(Y, N, zeta,D_v,D_h):
  prev_X = Y
  sum = np.zeros((N,N))
  for n in range(1,N1):
      for m in range(1,N1):
        slope2 = I + lamda*np.transpose((I - 0.25*(matrix_power(D_h,n) + matrix_power(D_v,n)) - 0.25*(matrix_power(D_h,m) + matrix_power(D_v,m))))@(I - 0.25*(matrix_power(D_h,n) + matrix_power(D_v,n)) - 0.25*(matrix_power(D_h,m) + matrix_power(D_v,m)))*alpha**((m+n)**2)
  for i in range(1,50):
    for n in range(1,N1):
        for m in range(1,N1):
          slope = prev_X - Y + lamda*np.transpose((I - 0.25*(matrix_power(D_h,n) + matrix_power(D_v,n)) - 0.25*(matrix_power(D_h,m) + matrix_power(D_v,m))))@( prev_X - 0.25*(matrix_power(D_h,n) + matrix_power(D_v,n))@prev_X - 0.25*prev_X@(matrix_power(D_h,m) + matrix_power(D_v,m)))*alpha**((m+n)**2)
    mu = np.linalg.pinv(zeta*I + np.diag(np.diag(slope2)))
    X = prev_X - mu@slope
    prev_X = X
  return prev_X

In [81]:
X2 = SD(Y,N,zeta,D_v,D_h)
X2 = SD(X2,N,zeta,D_h,D_v)
img2 = Image.fromarray((X2))
img2.show()

In [82]:
## computing MSE
Bilat_MSE = ((X_bi - Y)**2).mean(axis=None)
New_MSE = ((X2 - Y)**2).mean(axis=None)
MSE = ((X - Y)**2).mean(axis=None)

In [83]:
Bilat_MSE

2144.908005816398

In [84]:
New_MSE

1245.965294501479

In [85]:
MSE

6143.607522913572

In [86]:
100*((New_MSE - Bilat_MSE)/Bilat_MSE)

-41.910548558597135